In [102]:
pip install inflect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install textblob

In [ ]:
pip install compound-word-splitter


In [ ]:
pip install enchant

In [ ]:
pip install wordninja

In [ ]:
pip install pinecone

In [ ]:
pip install -U langchain-community

In [51]:
import requests
from bs4 import BeautifulSoup
import json

url = 'https://www.jurist.org/news/'

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    main = soup.find('div', class_='main-content')

    wrapper = main.find('div', class_='wrapper')
    main_wrapper = wrapper.find('section', class_='main-wrapper')
    posts = main.find('div', class_='post-list')

    blog_posts = main_wrapper.find_all('article', class_='blog-post')

    articles = {}
    title = ''
    for article in blog_posts:
        title_tag = article.find('strong', class_='ui-heading3 _no-margin').find('a')
        link = title_tag['href']
        title = title_tag['title'].strip('Permalink to ')
        author = article.find('div', class_='author').find('span', class_='name').get_text(strip=True).split('|')[0]
        post_date = article.find('div', class_='date').find('time', class_='post-date published')['datetime']

        article_url = link
        
        art_resp = requests.get(article_url)
        if art_resp.status_code == 200:
            art_soup = BeautifulSoup(art_resp.content, 'html.parser')
            art_main = art_soup.find('div', class_='main-content')
            art_wrapper = art_main.find('div', class_='wrapper')
            art_main_wrapper = art_wrapper.find('section', class_='main-wrapper')
            single_post = art_main_wrapper.find('div', class_='single-post')
            art_post = single_post.find('article', class_='blog-post')
            art_content = art_post.find('div', class_='content')
            art_body = art_content.find('div', class_='body _no-margin-bottom _no-padding-bottom')

            text = []
            paragraphs = art_body.find_all('p')
            for parag in paragraphs:
                text.append(parag.get_text(strip=True))
         
            content = ' '.join(text)
        articles[title] = { 'author': author.split('|')[0],
                            'post_date': post_date,
                            'link': link,
                            'content': content
                         }
        print(f"link: {link}")
        print(f"Title: {title}")
        print(f"Author: {author}")
        print(f"Post Date: {post_date}")
        print(f"Text: {content}")
        print("-" * 80)

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

pretty_json = json.dumps(articles, indent=4)
print(pretty_json)


link: https://www.jurist.org/news/2024/07/tunisia-court-issues-arrest-warrants-for-former-mp-and-gafsa-phosphate-company-ceo-in-corruption-case/
Title: Tunisia court issues arrest warrants for former MP and Gafsa Phosphate Company CEO in corruption cas
Author: Nova Kruijning 
Post Date: 2024-07-14T21:02:14-04:00
Text: The Tunis Court of Appeal Specializing in Financial Corruption Cases has issued arrest warrants against Tunisian businessman and former MP Lotfi Ali, as well as the former CEO of the Gafsa Phosphate Company (CPG), Romdhane Souid, TAPreportedSaturday. Speaking to TAP, Deputy Attorney General Sonia Moussaoui confirmed the issue of the arrest warrants, and in addition to naming Souid and Ali, identified three additional defendants; Slim Feriani, who served as the Minister of Industry under the 2016-2020 Chahed government, as well as Mouldi Ali, Lotfi’s brother, and Abdelwaheb Hfaïedh. Per Moussaoui’s statement, the warrants were issued in connection with alleged corruption c

In [112]:
articles['Japan defense ministry announces disciplinary action against over 200 members']['cleaned'][0]

'japan defens ministri announc disciplinari action friday two hundr and eighteen member ministri self defens forc variou misconduct includ mishandl classifi materi among disciplin eleven offici dismiss two devot eighty-thre suspend fourteen receiv pay cut seven formal reprimand remain personnel receiv admonish warn investig uncov sever inappropri activ mishandl classifi inform fraudul receipt allow power harass notabl fifty-eight case involv improp handl confidenti inform within grind maritim air self defens forc crew member sever maritim self defens forc ms destroy share sensit inform unqualifi colleagu addit sixty-two ms personnel includ senior member fraudul claim allow dive train duti amount approxim forty-thre million men two hundr and seventi zero twenti ms member offici improperli consum free meal base cafeteria power harass also signific issu three ministri offici director level higher accus make intimid remark subordin caus psycholog distress respons defens minist minor hard e

In [114]:
for k, v in articles.items():
    print(k)

Tunisia court issues arrest warrants for former MP and Gafsa Phosphate Company CEO in corruption cas
Tigray Interim Administration announces crackdown on escalating crime and security threats
d parliament rejects bill to decriminalize consensual abortion assistanc
US President Biden signs bill promoting distinct Tibet identity
China and Russia launch joint naval exercise amid NATO tensions
hilippines Senate orders arrest of mayor amid Chinese crime network allegations and identity controversy
stan Supreme Court overturns conviction of former PM Imran Khan in marriage cas
Japan defense ministry announces disciplinary action against over 200 members
Latvia constitutional court upholds law converting Russian language schools to Latv
Ukraine Prosecutor General urges ICC to open case against Russia for strike on children&#8217;s hosp


In [111]:
#Create dict of preprocessed articles
preproc = PreProcessNlp()

for k, v in articles.items():
    v['cleaned'] = []
    sentences = nltk.sent_tokenize(v['content'])
    
    for sentence in sentences:
        cleaned_sentance = preproc.preprocess(sentence,lemm=True,stem=False)
        v['cleaned'] = v['cleaned'] + cleaned_sentance
    v['cleaned'] = [" ".join(v['cleaned'])]

In [113]:
#Add embedding to dict
MODEL = "text-embedding-3-small"
client = openai.OpenAI(api_key = my_sk)

for k, v in articles.items():
    print(v['cleaned'])
    res = client.embeddings.create(
        input=[v['cleaned'][0]],
        model=MODEL
    )
    v['embedding'] = res.data

['tunic court appeal special financi corrupt case issu arrest warrant tunisian businessman former lot ali well former co ga sa phosphat compani pg ha ne id tap report saturday speak tap deputi attorney gener sonya mous oui confirm issu arrest warrant addit name id ali identifi three addit defend slim fera ni serv minist industri two thousand and sixteen two thousand and twenti ha govern well mould ali lot brother abel wa ed per mous oui statement warrant issu connect alleg corrupt crime involv contract extract transport phosphat id ali three defend phosphat use fertil often regard drive forc tunica mine sector howev also frequent sourc social tension routin face alleg corrupt august two thousand and twenty-on tunisian judg banner twelv govern offici travel due suspect involv corrupt activ relat mine transport phosphat individu later arrest charg corrupt crime year presid mai say ed vow crack corrupt industri give sector signific contribut tunica gdp phosphat mine account gdp fifteen ex

In [115]:
articles

{'Tunisia court issues arrest warrants for former MP and Gafsa Phosphate Company CEO in corruption cas': {'author': 'Nova Kruijning ',
  'post_date': '2024-07-14T21:02:14-04:00',
  'link': 'https://www.jurist.org/news/2024/07/tunisia-court-issues-arrest-warrants-for-former-mp-and-gafsa-phosphate-company-ceo-in-corruption-case/',
  'content': 'The Tunis Court of Appeal Specializing in Financial Corruption Cases has issued arrest warrants against Tunisian businessman and former MP Lotfi Ali, as well as the former CEO of the Gafsa Phosphate Company (CPG), Romdhane Souid, TAPreportedSaturday. Speaking to TAP, Deputy Attorney General Sonia Moussaoui confirmed the issue of the arrest warrants, and in addition to naming Souid and Ali, identified three additional defendants; Slim Feriani, who served as the Minister of Industry under the 2016-2020 Chahed government, as well as Mouldi Ali, Lotfi’s brother, and Abdelwaheb Hfaïedh. Per Moussaoui’s statement, the warrants were issued in connection 

In [96]:
import nltk
from nltk.tokenize import word_tokenize

input_text = "Welcome to Educative"
individual_words = word_tokenize(input_text)

print(individual_words)

['Welcome', 'to', 'Educative']


In [110]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
nltk.download('wordnet')

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import string

import re
import inflect

from textblob import TextBlob

import wordninja


class PreProcessNlp:
    def lower(self, sentence):
        lowercase_sentences = sentence.lower()
        return lowercase_sentences

    def split_words(self, sentence):
        split_words = [' '.join(wordninja.split(word)) for word in sentence.split()]
        return ' '.join(split_words)
        
    def rm_punctuation_re(self, sentence):
        #symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
        #for i in symbols:
        #    sentence = sentence.replace(i, ' ')
        tokens = word_tokenize(sentence)
        cleaned_tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
        return ' '.join(cleaned_tokens)

    def rm_punctuation_str(self, sentence):
        cleaned_sentence = sentence.translate(str.maketrans('', '', string.punctuation))
        return cleaned_sentence
    
    def remove_single_characters(self, sentence):
        words = sentence.split()
        new_text = ""
        for w in words:
            if len(w) > 1:
                new_text = new_text + " " + w
        return new_text
        
    def remove_stop_words(self, sentence):
        stop_words = set(stopwords.words('english'))
        #filtered_words = [f'{word}' for word in sentence.split() if word not in stop_words]
        filtered_words = [f'{word}' for word in word_tokenize(sentence) if word not in stop_words]
        filtered_text = ' '.join(filtered_words)
        return filtered_text
    
    def correct_words(self, sentence):
        blob = TextBlob(sentence)
        corrected_sentence = str(blob.correct())
        return corrected_sentence

    def num_to_word(self, sentence):
        p = inflect.engine()
        converted_text = []
        for word in sentence.split():
            if word.replace('.', '', 1).isdigit():  # Handle numbers with decimal points
                converted_text.append(p.number_to_words(word))
            else:
                converted_text.append(word)
        return ' '.join(converted_text)
    
    def stem(self, sentence):
        porter = PorterStemmer()
        words = sentence.split()
        new_text = ""
        for word in words:
            w = porter.stem(word)
            new_text = new_text + " " + w
        return new_text

    def lemm(self, sentence):
        lemmatizer = WordNetLemmatizer()
        lemmatized_words = [lemmatizer.lemmatize(word, pos="v") for word in sentence.split()]     
        return lemmatized_words
    
    def preprocess(self, sentencem, lemm=False, stem=False):
        i = self.lower(sentence)
        #print(f'lowered sentance:{i}')
        i = self.split_words(i)
        i = self.correct_words(i)
        #print(f'corrected words{i}')
        i = self.rm_punctuation_str(i)
        #print(f'removed puntuation:{i}')
        i = self.remove_single_characters(i)
        #print(f'removed single chars{i}')
        i = self.remove_stop_words(i)
        #print(f'removed stop words:{i}')
        i = self.num_to_word(i)
        if self.stem:
            i = self.stem(i)
        if self.lemm:
            i = self.lemm(i)
        return i

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
import openai
def generate_embeddings(text, my_sk):
    client = openai.OpenAI(api_key = my_sk)
    
    response = client.embeddings.create(
        input=[text],
        model="text-embedding-3-small"
    )
    
    return response.data

In [58]:
import time
from pinecone import ServerlessSpec

from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_key)
spec = ServerlessSpec(cloud="aws", region="us-east-1")

index_name = 'semantic-search-openai'

# check if index already exists (if shouldn't if this is your first run)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=len(articles['Permalink to Pakistan Supreme Court overturns conviction of former PM Imran Khan in marriage case']['embedding'][0].embedding),  # dimensionality of text-embed-3-small
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10}},
 'total_vector_count': 10}

In [ ]:
for k, v in articles.items():
    #print(v['embedding'][0].embedding)
    print({"metadata": {"text": v['cleaned'][0]}})

In [59]:
#Update article vectors DB
id = 11
article_vectors = []
for k, v in articles.items():
    article_vectors.append({"id": str(id), "values": v['embedding'][0].embedding, "metadata": {"text": v['cleaned'][0]}})
    id += 1

upsert_response = index.upsert(
    vectors=article_vectors)


In [42]:
def handle_query(query, client, index_name):
    try:
        index = pc.Index(index_name)
        emb = client.embeddings.create(input=query, model=MODEL).data[0].embedding
        retrived_text = index.query(vector=[emb], top_k=2, include_metadata=True)
        output = [match['metadata']['text'] for match in retrived_text["matches"]]
        return "\n".join(output)
    except Exception as e:
        raise e

In [116]:
query = "news about Latvian legal framework"
source_resp = handle_query(query, client, index_name)

In [48]:
#Get Response from OpenAI

from openai import OpenAI
from dotenv import load_dotenv
from dataclasses import dataclass

load_dotenv()

class AIResponse:
    def __init__(self, userResponse, source_knowledge):
        self.userResponse = userResponse
        self.source_knowledge = source_knowledge

    def generatePrompt(self):
        try:
            aug_prompt = f"""
            You are a chatbot that is trained to answer for user response.
            You are given the following context:
            {self.source_knowledge}
            You are asked to generate short and accurate answer to the following question using above context.
            question:
            {self.userResponse}
            strictly do not hallucinate. only use the above context to generate an answer.
            """
            return aug_prompt

        except Exception as e:
            raise e
    
    def generateResponse(self):
        try:
            prompt = self.generatePrompt()
            response = client.chat.completions.create(
                model="gpt-4-1106-preview",
                max_tokens=1024,
                temperature=0,
                messages= [
                    {"role": "system", "content": prompt}
                ]
            )
            model_response = response.choices[0].message.content 
            return model_response
        
        except Exception as e:
            raise e

In [61]:
model = AIResponse(query, source_resp)
model_response = model.generateResponse()

In [117]:
model_response

"The Constitutional Court of Latvia ruled against a challenge to a law that converts Russian-language schools in Latvia to state that the decision to make Latvian the language of instruction in schools is valid within the country's legal framework and international commitments. Latvia, a former member of the Soviet Union that gained independence and officially became a country in 1991, is liable to its Russian-speaking population, which is a major part of the population. Article 4 of the Constitution of Latvia recognizes Latvian as the official language of the nation, and Article 114 gives the right to ethnic minorities to preserve their language. The Education Law of 1998 and the General Education Law of 2000 require public schools to use Latvian as the primary language of instruction, while private institutions can choose to teach in Russian. In 2018, the amendments to the General Education Law were passed, making teaching in Latvian compulsory in all educational institutions, includ

In [ ]:
#Genereate tags for content using spacy
Import spacy
import json

# Load English language model
nlp = spacy.load(“en_core_web_sm”)

# Sample text with meta data candidates
text = “””In a blog post titled ‘The Top 10 Tech Trends of 2024,’ 
John Doe discusses the rise of artificial intelligence and machine learning 
in various industries. The article mentions companies like Google and Microsoft 
as pioneers in AI research. Additionally, it highlights emerging technologies 
such as natural language processing and computer vision.”””

# Process the text with spaCy
doc = nlp(text)

# Extract named entities and their labels
meta_data = [{“text”: ent.text, “label”: ent.label_} for ent in doc.ents]

# Convert meta data to JSON format
meta_data_json = json.dumps(meta_data)

print(meta_data_json)

# output
“””
[
    {“text”: “2024”, “label”: “DATE”},
    {“text”: “John Doe”, “label”: “PERSON”},
    {“text”: “Google”, “label”: “ORG”},
    {“text”: “Microsoft”, “label”: “ORG”},
    {“text”: “AI”, “label”: “ORG”},
    {“text”: “natural language processing”, “label”: “ORG”},
    {“text”: “computer vision”, “label”: “ORG”}
]
“””